In [37]:
import pandas as pd

df=pd.read_excel(r'/Users/gilbenor/Documents/code projects/msc/counting_research_algorithms/fifty_one/measurements/updated_filtered_data_with_lengths_body-all.xlsx')

print(len(df))
df=df.dropna()
print(len(df))

# df['Pond_Type'] = df['Pond_Type'].replace({'test-car': 'square-femal', 'test-right': 'circular big male', 'test-left': 'circular small male'})
df['Pond_Type'].unique()

132
117


array(['right', 'left', 'car'], dtype=object)

In [38]:
# First, let's see what unique values we have before replacement
print("Before replacement:", df['Pond_Type'].unique())

# Then do a complete replacement that covers all cases
df['Pond_Type'] = df['Pond_Type'].replace({
    'car': 'square',
    # 'test-right': 'circle2',
    'right': 'circle_female',  # Added this to handle 'right'
    'left': 'circle_male',
    # 'test-left': 'circle1'
})

print("After replacement:", df['Pond_Type'].unique())

Before replacement: ['right' 'left' 'car']
After replacement: ['circle_female' 'circle_male' 'square']


In [39]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import pearsonr, spearmanr, shapiro, levene
# import pingouin as pg
import statsmodels.api as sm
import numpy as np
import plotly.express as px
def calculate_mape(estimated_lengths, true_lengths):
    """
    Calculate Mean Absolute Percentage Error
    
    Parameters:
    -----------
    estimated_lengths : array-like
        Estimated length measurements (Len_e)
    true_lengths : array-like
        True length measurements (Len_t)
        
    Returns:
    --------
    float
        Mean Absolute Percentage Error (%)
    """
    # Calculate individual absolute percentage errors
    absolute_percentage_errors = [abs(est - true) / true * 100 for est, true in zip(estimated_lengths, true_lengths)]
    
    # Calculate mean of absolute percentage errors
    
    return absolute_percentage_errors
# Load your data

# Calculate expert averages and standard deviations
df['expert_mean'] = df[['Length_1', 'Length_2', 'Length_3']].median(axis=1)
df['expert_std'] = df[['Length_1', 'Length_2', 'Length_3']].std(axis=1)

df['expert_max'] = df[['Length_1', 'Length_2', 'Length_3']].max(axis=1)
df['expert_min'] = df[['Length_1', 'Length_2', 'Length_3']].min(axis=1)

#diffrence in pixels between the min and max of Length_1_pixels, Length_2_pixels, Length_3_pixels

df['diff_max_min'] =abs(df[['Length_1', 'Length_2', 'Length_3']].max(axis=1) - df[['Length_1', 'Length_2', 'Length_3']].min(axis=1))

print(f'mean difference between min max' ,df['diff_max_min'].mean())
print(f'std  difference between min max ',df['diff_max_min'].std())
#
median_diff_max_min = df['diff_max_min'].median()
print(f'median difference between min max', median_diff_max_min)

#violoin plot of the difference between the min and max of the three measurements
max_diff_threshold = 35  # Adjust this value based on your needs

# Filter out rows with too large differences
df_filtered = df[df['diff_max_min'] <= max_diff_threshold]

# Create violin plot with filtered data
fig = px.violin(df_filtered,
                x='Pond_Type', 
                y='diff_max_min', 
                box=True,  # Include box plot
                points="all",  # Show all points
                hover_data=['Label', 'PrawnID'],  # Include hover data
                title=f'Violin plot of diff_max_min in mm (threshold: {max_diff_threshold}mm)')

# Print statistics about filtered data
print(f"Original data points: {len(df)}")
print(f"Filtered data points: {len(df_filtered)}")
print(f"Removed {len(df) - len(df_filtered)} outliers ({((len(df) - len(df_filtered))/len(df))*100:.1f}%)")

fig.show()

#how many Length_ground_truth_annotation(mm) in comparison with lengthe_1, length_2, length_3 bigger than 50%

# df['difference_mean_annotation_expert'] = abs(df['Length_ground_truth_annotation(mm)'] - df['expert_mean']/df['Length_ground_truth_annotation(mm)']) 
# df['difference_max_annotation_expert'] = abs(df['Length_ground_truth_annotation(mm)'] - df['expert_max']/df['Length_ground_truth_annotation(mm)'])
# df['difference_min_annotation_expert'] = abs(df['Length_ground_truth_annotation(mm)'] - df['expert_min']/df['Length_ground_truth_annotation(mm)'])

# df['min difference'] = df[['difference_mean_annotation_expert', 'difference_max_annotation_expert', 'difference_min_annotation_expert']].min(axis=1)
# df['max_difference'] = df[['difference_mean_annotation_expert', 'difference_max_annotation_expert', 'difference_min_annotation_expert']].max(axis=1)    


# df['difference_max_annotation_expert'] = abs(df['Length_ground_truth_annotation(mm)'] - df['expert_max']/df['Length_ground_truth_annotation(mm)'])

# df['difference_min_annotation_expert'] = abs(df['Length_ground_truth_annotation(mm)'] - df['expert_min']/df['Length_ground_truth_annotation(mm)'])
#violin difference_annotation_expert
# fig = px.violin(df,
#                 x='Pond_Type',
#                 y='min difference',
#                 box=True,  # Include box plot
#                 points="all",  # Show all points
#                 hover_data=['Label', 'PrawnID'],  # Include hover data
#                 title=f'Violin plot of difference_annotation_expert',)
# #Save the plot to png
# fig.write_image(f'violin carapace difference_annotation_expert.png')
# fig.show()




# Calculate differences
# df['diff_mean'] =abs(df['Length_fov(mm)'] - df['expert_mean'])
# df['diff_max'] =abs(df['Length_fov(mm)'] - df['expert_max'])
# df['diff_min'] =abs(df['Length_fov(mm)'] - df['expert_min'])

# df['the_min_diff'] = df[['diff_mean', 'diff_max', 'diff_min']].min(axis=1)


#the length measurement of the the_min_diff
# df['the_min_diff_length'] =df['Length_fov(mm)'] - df['the_min_diff']

#the length measurement of the the_max_diff
#percent error of the the_min_diff


# df['org_min_diff'] = [
#     df['Length_fov(mm)'].iloc[i] - df['the_min_diff'].iloc[i] 
#     if df['the_min_diff'].iloc[i] > 0 
#     else df['Length_fov(mm)'].iloc[i] + abs(df['the_min_diff'].iloc[i]) 
#     for i in range(len(df))
# ]


# df['min difference']=df[['Error_percentage_distance_mm_ground_min', 'Error_percentage_distance_mm_ground_max','Error_percentage_distance_mm_ground_median']].min(axis=1)

# Calculate the percentage error
# df['the_min_diff_percentage_error'] = (df['the_min_diff'].abs() / df['org_min_diff'].abs()) * 100

# # Cap the error at 100%
# df['the_min_diff_percentage_error'] = df['the_min_diff_percentage_error'].apply(lambda x: min(x, 100))

# the percentage error of the the_min_diff when smaller than 100%
# df['the_min_diff_percentage_error'] = df['the_min_diff_percentage_error']
# df['min difference']=df[['Error_percentage_distance_mm_ground_min', 'Error_percentage_distance_mm_ground_max','Error_percentage_distance_mm_ground_median']].min(axis=1)

# Calculate the percentage error
# df['the_min_diff_percentage_error'] = (df['the_min_diff'].abs() / df['org_min_diff'].abs()) * 100

# # Cap the error at 100%
# df['the_min_diff_percentage_error'] = df['the_min_diff_percentage_error'].apply(lambda x: min(x, 100))

# the percentage error of the the_min_diff when smaller than 100%
# df['the_min_diff_percentage_error'] = df['the_min_diff_percentage_error']
df['min_mpe']=df[['MPError_fov_min', 'MPError_fov_max','MPError_fov_median']].min(axis=1)


#remove the outliers of the min_mpe
df_filtered = df[df['min_mpe'] <= 50]





df['max_mpe']=df[['MPError_fov_min', 'MPError_fov_max','MPError_fov_median']].max(axis=1)

df['min_mae']=df[['AbsError_fov_min', 'AbsError_fov_max','AbsError_fov_median']].min(axis=1)

df['max_mae']=df[['AbsError_fov_min', 'AbsError_fov_max','AbsError_fov_median']].max(axis=1)

#remove the outliers of the min_mae
df_filtered = df[df['min_mae'] <= 50]  




#differente between pred pixels and expert measurement pixels
df['pred_pixels_diff_1']=abs( df['pred_Distance_pixels'] - df['Length_1_pixels'])
df['pred_pixels_diff_2']=abs(df['pred_Distance_pixels'] - df['Length_2_pixels'])
df['pred_pixels_diff_3']=abs(df['pred_Distance_pixels'] - df['Length_3_pixels'])


#min error in pixels
df['min_error_pixels']=df[['pred_pixels_diff_1', 'pred_pixels_diff_2', 'pred_pixels_diff_3']].min(axis=1)



#mape in pixels 
df['mape_pixels_1']=calculate_mape(df['pred_Distance_pixels'], df['Length_1_pixels'])
df['mape_pixels_2']=calculate_mape(df['pred_Distance_pixels'], df['Length_2_pixels'])
df['mape_pixels_3']=calculate_mape(df['pred_Distance_pixels'], df['Length_3_pixels'])


#min mape in pixels
df['min_mape_pixels']=df[['mape_pixels_1', 'mape_pixels_2', 'mape_pixels_3']].min(axis=1)


#print the median of the  error in pixels
print('median of the error in pixels:')
print(df['min_error_pixels'].mean ())

#print the median of the mape in pixels
print('median of the mape in pixels:')
print(df['min_mape_pixels'].mean())


#print for each pond type the min of the error in pixels and min
print('min  of the error in pixels for each pond type:')
for pond_type in df['Pond_Type'].unique():
    print(f'{pond_type}:')
    print(f'min error pixels: {df[df["Pond_Type"]==pond_type]["min_error_pixels"].min():.2f}')
    print(f'median mape pixels: {df[df["Pond_Type"]==pond_type]["min_mape_pixels"].min():.2f}%')
    print()


print('max mae median:')
print(df.groupby('Pond_Type')['max_mae'].median())

#max mpe median
print('max mpe median:')
print(df.groupby('Pond_Type')['max_mpe'].median())


# Create a violin plot using plotly
import plotly.express as px
#violin plots wiht plotly for mean percentage error and MAE
for error in ['min_mpe', 'min_mae', 'max_mpe', 'max_mae']:
        # Create a violin plot using plotly 
    fig = px.violin(df,
                    x='Pond_Type', 
                    y=error, 
                    box=True,  # Include box plot
                    points="all",  # Show all points
                    hover_data=['Label', 'PrawnID'],  # Include hover data
                    title=f'Violin plot of {error} in mm' )
    #Save the plot to png
    # fig.write_image(f'violin full {error}_.png')
    fig.show()

#change name of each pond type if test-car change to square-female if test-right change to circular big male if test-left change to circular small mal

print('min mae median:')
print("min mae for all pond types:")
print(df['min_mae'].median())
print("min mae for each pond type:")
print(df.groupby('Pond_Type')['min_mae'].median())

#std of min mae
print('std of min mae:')
print(df.groupby('Pond_Type')['min_mae'].std())

#std of min mpe
print('std of min mpe:')
print(df.groupby('Pond_Type')['min_mpe'].std())




#min mpe median
print('min mpe median:')
print("min mpe for all pond types:")
print(df['min_mpe'].median())
print("min mpe for each pond type:")
print(df.groupby('Pond_Type')['min_mpe'].median())


# #scatter plot the_min_diff_percentage_error and  difference_annotation_expert plotly
# fig = px.scatter(df, x='min_mpe', y='min difference', color='Pond_Type', hover_data=['Label', 'PrawnID'])
# #add title
# fig.update_layout(title='min_mpe vs. difference_annotation_expert')

# #save the plot to png
# fig.write_image('the_min_diff_percentage_error vs. difference_annotation_expert.png')
# # Add a trendline


# fig.show()


# #scatter plot the_min_diff_percentage_error and  pose_eval_iou plotly
# fig = px.scatter(df, x='min_mpe', y='pose_eval_iou', color='Pond_Type', hover_data=['Label', 'PrawnID'])
# #add title
# fig.update_layout(title='min_mpe vs. pose_eval_iou')

# #save the plot to png
# fig.write_image('the_min_diff_percentage_error vs. pose_eval_iou.png')
# # Add a trendline
# fig.show()

#scatter plot between min_mpe and height
fig = px.scatter(df, x='Height(mm)', y='min_mpe', hover_data=['Label', 'PrawnID'])
#add title
fig.update_layout(title='min_mpe vs. Height')

#save the plot to png
# fig.write_image('full body_ min_mpe vs. Height.png')
# Add a trendline
fig.show()

#max mae median

#total min mae


# Simple analysis of pixel-to-length conversion error using predicted pixels
print("\n----- Pixel to Length Conversion Error Analysis -----\n")

# Calculate the actual conversion factor (mm/pixel) for each expert measurement
df['expert_conv_factor_1'] = df['Length_1'] / df['Length_1_pixels']
df['expert_conv_factor_2'] = df['Length_2'] / df['Length_2_pixels']
df['expert_conv_factor_3'] = df['Length_3'] / df['Length_3_pixels']

# Calculate median expert conversion factor for each prawn (more robust than mean)
df['expert_median_conv_factor'] = df[['expert_conv_factor_1', 'expert_conv_factor_2', 'expert_conv_factor_3']].median(axis=1)

# Calculate the algorithm's conversion factor (using the predicted measurements)
df['algorithm_conv_factor'] = df['Length_fov(mm)'] / df['pred_Distance_pixels']

# Calculate the difference between expert and algorithm conversion factors
df['conv_factor_diff'] = abs(df['expert_median_conv_factor'] - df['algorithm_conv_factor'])
df['conv_factor_diff_pct'] = (df['conv_factor_diff'] / df['expert_median_conv_factor']) * 100

# Calculate what the length would be if using expert conversion factor with predicted pixels
df['length_using_expert_factor'] = df['pred_Distance_pixels'] * df['expert_median_conv_factor']

# Calculate error due to conversion factor difference
df['conversion_error_mm'] = abs(df['length_using_expert_factor'] - df['Length_fov(mm)'])
df['conversion_error_pct'] = (df['conversion_error_mm'] / df['Length_fov(mm)']) * 100

# Print summary statistics
print("Summary statistics for pixel-to-length conversion:")
print(f"Mean expert conversion factor: {df['expert_median_conv_factor'].mean():.4f} mm/pixel")
print(f"Mean algorithm conversion factor: {df['algorithm_conv_factor'].mean():.4f} mm/pixel")
print(f"Mean absolute difference in conversion factors: {df['conv_factor_diff'].mean():.4f} mm/pixel ({df['conv_factor_diff_pct'].mean():.2f}%)")
print(f"Mean error due to conversion factor difference: {df['conversion_error_mm'].mean():.2f} mm ({df['conversion_error_pct'].mean():.2f}%)")

# Print stats by pond type
print("\nConversion error by pond type:")
for pond_type in df['Pond_Type'].unique():
    pond_df = df[df['Pond_Type'] == pond_type]
    print(f"\n{pond_type} Pond:")
    print(f"  Expert conversion factor: {pond_df['expert_median_conv_factor'].mean():.4f} mm/pixel")
    print(f"  Algorithm conversion factor: {pond_df['algorithm_conv_factor'].mean():.4f} mm/pixel")
    print(f"  Conversion factor difference: {pond_df['conv_factor_diff'].mean():.4f} mm/pixel ({pond_df['conv_factor_diff_pct'].mean():.2f}%)")
    print(f"  Error due to conversion: {pond_df['conversion_error_mm'].mean():.2f} mm ({pond_df['conversion_error_pct'].mean():.2f}%)")

# Calculate what percentage of the total error is due to conversion issues
df['pixel_pred_error'] = abs(df['pred_Distance_pixels'] - df[['Length_1_pixels', 'Length_2_pixels', 'Length_3_pixels']].mean(axis=1))
df['pixel_pred_error_pct'] = (df['pixel_pred_error'] / df[['Length_1_pixels', 'Length_2_pixels', 'Length_3_pixels']].mean(axis=1)) * 100

print("\nContribution of conversion error to total error:")
print(f"Mean error in pixel prediction: {df['pixel_pred_error_pct'].mean():.2f}%")
print(f"Mean error in conversion factor: {df['conv_factor_diff_pct'].mean():.2f}%")
print(f"Mean total percentage error: {df['min_mpe'].mean():.2f}%")

# Calculate approximate contribution
conversion_contribution = (df['conversion_error_pct'].mean() / df['min_mpe'].mean()) * 100
print(f"Conversion error contributes approximately {conversion_contribution:.1f}% of the total error")
# Normality Test


# Function to calculate MAD manually
def calculate_mad(series):
    median_val = series.median()
    return (series - median_val).abs().median()

print("\n----- Error Ranges for Paper (using manual MAD) -----\n")

# Calculate medians and MADs for overall data
min_mpe_median = df['min_mpe'].median()
max_mpe_median = df['max_mpe'].median()
min_mae_median = df['min_mae'].median()
max_mae_median = df['max_mae'].median()

min_mpe_mad = calculate_mad(df['min_mpe'])
max_mpe_mad = calculate_mad(df['max_mpe'])
min_mae_mad = calculate_mad(df['min_mae'])
max_mae_mad = calculate_mad(df['max_mae'])

print("Overall error ranges (median ± MAD):")
print(f"Percentage error: {min_mpe_median:.1f}% ± {min_mpe_mad:.1f}% to {max_mpe_median:.1f}% ± {max_mpe_mad:.1f}%")
print(f"Absolute error: {min_mae_median:.1f}mm ± {min_mae_mad:.1f}mm to {max_mae_median:.1f}mm ± {max_mae_mad:.1f}mm")

# By pond type
print("\nBy pond type:")
for pond_type in df['Pond_Type'].unique():
    pond_df = df[df['Pond_Type'] == pond_type]
    n_samples = len(pond_df)
    
    # Calculate pond-specific medians and MADs
    pond_min_mpe_median = pond_df['min_mpe'].median()
    pond_max_mpe_median = pond_df['max_mpe'].median()
    pond_min_mae_median = pond_df['min_mae'].median()
    pond_max_mae_median = pond_df['max_mae'].median()
    
    pond_min_mpe_mad = calculate_mad(pond_df['min_mpe'])
    pond_max_mpe_mad = calculate_mad(pond_df['max_mpe'])
    pond_min_mae_mad = calculate_mad(pond_df['min_mae'])
    pond_max_mae_mad = calculate_mad(pond_df['max_mae'])
    
    print(f"\n{pond_type} Pond (n={n_samples}):")
    print("Median ± MAD ranges:")
    print(f"Percentage error: {pond_min_mpe_median:.1f}% ± {pond_min_mpe_mad:.1f}% to "
          f"{pond_max_mpe_median:.1f}% ± {pond_max_mpe_mad:.1f}%")
    print(f"Absolute error: {pond_min_mae_median:.1f}mm ± {pond_min_mae_mad:.1f}mm to "
          f"{pond_max_mae_median:.1f}mm ± {pond_max_mae_mad:.1f}mm")

# Format for paper
print("\nSuggested text for paper:")
print(f"The algorithm's predictions showed median measurement errors ranging from "
      f"{min_mpe_median:.1f}% ± {min_mpe_mad:.1f}% to {max_mpe_median:.1f}% ± {max_mpe_mad:.1f}% "
      f"({min_mae_median:.1f}mm ± {min_mae_mad:.1f}mm to {max_mae_median:.1f}mm ± {max_mae_mad:.1f}mm) "
      f"when compared against expert measurements.")

# Add pond-specific text
pond_texts = []
for pond_type in df['Pond_Type'].unique():
    pond_df = df[df['Pond_Type'] == pond_type]
    
    # Calculate pond-specific values
    pond_min_mpe_median = pond_df['min_mpe'].median()
    pond_max_mpe_median = pond_df['max_mpe'].median()
    pond_min_mae_median = pond_df['min_mae'].median()
    pond_max_mae_median = pond_df['max_mae'].median()
    
    pond_min_mpe_mad = calculate_mad(pond_df['min_mpe'])
    pond_max_mpe_mad = calculate_mad(pond_df['max_mpe'])
    pond_min_mae_mad = calculate_mad(pond_df['min_mae'])
    pond_max_mae_mad = calculate_mad(pond_df['max_mae'])
    
    pond_texts.append(
        f"{pond_type.lower()} ponds showed ranges of "
        f"{pond_min_mpe_median:.1f}% ± {pond_min_mpe_mad:.1f}% to "
        f"{pond_max_mpe_median:.1f}% ± {pond_max_mpe_mad:.1f}% "
        f"({pond_min_mae_median:.1f}mm ± {pond_min_mae_mad:.1f}mm to "
        f"{pond_max_mae_median:.1f}mm ± {pond_max_mae_mad:.1f}mm)"
    )

print("\nFor individual pond types: " + ", ".join(pond_texts) + ".")



import plotly.graph_objects as go

# Colors for different error ranges
colors = {'<5%': '#2ecc71', '5-10%': '#f1c40f', '>10%': '#e74c3c'}

# Create separate figure for each pond type
for pond_type in df['Pond_Type'].unique():
    # Filter data for this pond type
    pond_df = df[df['Pond_Type'] == pond_type].copy()
    
    # Create categories for coloring
    pond_df['error_category'] = pd.cut(
        pond_df['min_mpe'], 
        bins=[0, 5, 10, float('inf')],
        labels=['<5%', '5-10%', '>10%']
    )
    
    # Create figure
    fig = go.Figure()
    
    # For each error category
    for category in ['<5%', '5-10%', '>10%']:
        cat_df = pond_df[pond_df['error_category'] == category]
        if len(cat_df) > 0:
            # Sort by error value
            cat_df = cat_df.sort_values(by='min_mpe')
            
            # Create y-values to spread points vertically
            y_values = np.linspace(0.1, 0.9, len(cat_df))
            
            # Add scatter points
            fig.add_trace(
                go.Scatter(
                    x=cat_df['min_mpe'],
                    y=y_values,
                    mode='markers',
                    marker=dict(
                        color=colors[category],
                        size=10,
                        line=dict(width=1, color='black')
                    ),
                    name=category,
                    hovertemplate=
                    "ID: %{customdata[0]}<br>" + 
                    "Image: %{customdata[1]}<br>" +
                    "Error: %{x:.1f}%<br>" + 
                    "<extra></extra>",
                    customdata=cat_df[['PrawnID', 'Label']].values
                )
            )
    
    # Add vertical lines at 5% and 10% boundaries
    fig.add_shape(
        type="line", x0=5, x1=5, y0=0, y1=1,
        line=dict(color="gray", width=1, dash="dash")
    )
    fig.add_shape(
        type="line", x0=10, x1=10, y0=0, y1=1,
        line=dict(color="gray", width=1, dash="dash")
    )
    
    # Add annotations
    fig.add_annotation(
        x=5, y=0.95, text="5%", showarrow=False
    )
    fig.add_annotation(
        x=10, y=0.95, text="10%", showarrow=False
    )

    # Update layout
    fig.update_layout(
        height=500,
        width=800,
        title_text=f"Error Distribution - {pond_type} Pond",
        template="plotly_white",
        xaxis_title="Min MPE (%)",
        yaxis_title="",
        yaxis_showticklabels=False,
        xaxis_range=[0, df['min_mpe'].max() * 1.1],
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="center",
            x=0.5
        )
    )

    # Display the figure
    fig.show()


mean difference between min max 13.854444444444443
std  difference between min max  8.694703126383253
median difference between min max 12.359000000000009
Original data points: 117
Filtered data points: 115
Removed 2 outliers (1.7%)


median of the error in pixels:
60.49962200470192
median of the mape in pixels:
10.087029612449573
min  of the error in pixels for each pond type:
circle_female:
min error pixels: 0.16
median mape pixels: 0.03%

circle_male:
min error pixels: 1.99
median mape pixels: 0.54%

square:
min error pixels: 0.83
median mape pixels: 0.10%

max mae median:
Pond_Type
circle_female    18.473935
circle_male      18.353723
square           21.182030
Name: max_mae, dtype: float64
max mpe median:
Pond_Type
circle_female    16.338081
circle_male      29.269808
square           18.540554
Name: max_mpe, dtype: float64


min mae median:
min mae for all pond types:
6.161982936181815
min mae for each pond type:
Pond_Type
circle_female    5.651378
circle_male      8.743150
square           4.882152
Name: min_mae, dtype: float64
std of min mae:
Pond_Type
circle_female    85.261509
circle_male       5.996261
square            5.022200
Name: min_mae, dtype: float64
std of min mpe:
Pond_Type
circle_female    84.243289
circle_male      10.119902
square            4.967435
Name: min_mpe, dtype: float64
min mpe median:
min mpe for all pond types:
5.140557183125971
min mpe for each pond type:
Pond_Type
circle_female     4.767052
circle_male      12.893451
square            4.604901
Name: min_mpe, dtype: float64



----- Pixel to Length Conversion Error Analysis -----

Summary statistics for pixel-to-length conversion:
Mean expert conversion factor: 0.1702 mm/pixel
Mean algorithm conversion factor: 0.1846 mm/pixel
Mean absolute difference in conversion factors: 0.0168 mm/pixel (10.10%)
Mean error due to conversion factor difference: 11.30 mm (8.95%)

Conversion error by pond type:

circle_female Pond:
  Expert conversion factor: 0.1735 mm/pixel
  Algorithm conversion factor: 0.1868 mm/pixel
  Conversion factor difference: 0.0150 mm/pixel (9.04%)
  Error due to conversion: 11.14 mm (8.04%)

circle_male Pond:
  Expert conversion factor: 0.1778 mm/pixel
  Algorithm conversion factor: 0.2103 mm/pixel
  Conversion factor difference: 0.0326 mm/pixel (18.31%)
  Error due to conversion: 13.70 mm (15.37%)

square Pond:
  Expert conversion factor: 0.1290 mm/pixel
  Algorithm conversion factor: 0.1284 mm/pixel
  Conversion factor difference: 0.0110 mm/pixel (8.44%)
  Error due to conversion: 9.39 mm (8.38%

In [41]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go

# Load the data
# Remove rows where pose_eval_iou is below 0.85
if 'pose_eval_iou' in df.columns:
    print(f"Number of rows before filtering: {len(df)}")
    df = df[df['pose_eval_iou'] >= 0.85]
    print(f"Number of rows after filtering: {len(df)}")

# Check if pose_eval exists in the dataframe, print available columns if not
if 'pose_eval' not in df.columns and 'pose_eval_iou' not in df.columns:
    print("Warning: No 'pose_eval' or 'pose_eval_iou' column found in dataframe.")
    print(f"Available columns: {df.columns.tolist()}")

# Get the pose evaluation column name if it exists
pose_col = 'pose_eval' if 'pose_eval' in df.columns else ('pose_eval_iou' if 'pose_eval_iou' in df.columns else None)

# Colors for points based on pose_eval value
pose_colors = {'good_pose': 'blue', 'low_pose': 'red'}

# Create separate figure for each pond type
for pond_type in df['Pond_Type'].unique():
    # Filter data for this pond type
    pond_df = df[df['Pond_Type'] == pond_type].copy()
    
    # Create figure
    fig = go.Figure()
    
    # Create error categories for reference
    pond_df['error_category'] = pd.cut(
        pond_df['min_mpe'], 
        bins=[0, 5, 10, float('inf')],
        labels=['<5%', '5-10%', '>10%']
    )
    
    # Sort by error value
    pond_df = pond_df.sort_values(by='min_mpe')
    
    # Create y-values to spread points vertically
    y_values = np.linspace(0.1, 0.9, len(pond_df))
    
    if pose_col is not None:
        # Split data based on pose_eval
        low_pose_df = pond_df[pond_df[pose_col] < 0.85]
        good_pose_df = pond_df[pond_df[pose_col] >= 0.85]
        
        # Add trace for points with low pose_eval
        if len(low_pose_df) > 0:
            # Get indices for y values
            low_indices = [list(pond_df.index).index(idx) for idx in low_pose_df.index]
            low_y_values = [y_values[i] for i in low_indices]
            
            fig.add_trace(
                go.Scatter(
                    x=low_pose_df['min_mpe'],
                    y=low_y_values,
                    mode='markers',
                    marker=dict(
                        color=pose_colors['low_pose'],
                        size=10,
                        line=dict(width=1, color='black'),
                        symbol='circle'
                    ),
                    name=f'{pose_col} < 0.85',
                    hovertemplate=
                    "ID: %{customdata[0]}<br>" + 
                    "Image: %{customdata[1]}<br>" +
                    "Error: %{x:.1f}%<br>" + 
                    f"{pose_col}: %{{customdata[2]:.2f}}<br>" +
                    "Error category: %{customdata[3]}<br>" +
                    "<extra></extra>",
                    customdata=low_pose_df[['PrawnID', 'Label', pose_col, 'error_category']].values
                )
            )
        
        # Add trace for points with good pose_eval
        if len(good_pose_df) > 0:
            # Get indices for y values
            good_indices = [list(pond_df.index).index(idx) for idx in good_pose_df.index]
            good_y_values = [y_values[i] for i in good_indices]
            
            fig.add_trace(
                go.Scatter(
                    x=good_pose_df['min_mpe'],
                    y=good_y_values,
                    mode='markers',
                    marker=dict(
                        color=pose_colors['good_pose'],
                        size=10,
                        line=dict(width=1, color='black'),
                        symbol='circle'
                    ),
                    name=f'{pose_col} ≥ 0.85',
                    hovertemplate=
                    "ID: %{customdata[0]}<br>" + 
                    "Image: %{customdata[1]}<br>" +
                    "Error: %{x:.1f}%<br>" + 
                    f"{pose_col}: %{{customdata[2]:.2f}}<br>" +
                    "Error category: %{customdata[3]}<br>" +
                    "<extra></extra>",
                    customdata=good_pose_df[['PrawnID', 'Label', pose_col, 'error_category']].values
                )
            )
    else:
        # If no pose evaluation column exists, show all points in gray
        fig.add_trace(
            go.Scatter(
                x=pond_df['min_mpe'],
                y=y_values,
                mode='markers',
                marker=dict(
                    color='gray',
                    size=10,
                    line=dict(width=1, color='black')
                ),
                name='All points (pose_eval not available)',
                hovertemplate=
                "ID: %{customdata[0]}<br>" + 
                "Image: %{customdata[1]}<br>" +
                "Error: %{x:.1f}%<br>" + 
                "Error category: %{customdata[2]}<br>" +
                "<extra></extra>",
                customdata=pond_df[['PrawnID', 'Label', 'error_category']].values
            )
        )
    
    # Add vertical lines at 5% and 10% boundaries
    fig.add_shape(
        type="line", x0=5, x1=5, y0=0, y1=1,
        line=dict(color="gray", width=1, dash="dash")
    )
    fig.add_shape(
        type="line", x0=10, x1=10, y0=0, y1=1,
        line=dict(color="gray", width=1, dash="dash")
    )
    
    # Add annotations
    fig.add_annotation(x=5, y=0.95, text="5%", showarrow=False)
    fig.add_annotation(x=10, y=0.95, text="10%", showarrow=False)

    # Update layout
    fig.update_layout(
        height=500,
        width=800,
        title_text=f"Error Distribution by Pose Evaluation - {pond_type} Pond",
        template="plotly_white",
        xaxis_title="Min MPE (%)",
        yaxis_title="",
        yaxis_showticklabels=False,
        xaxis_range=[0, df['min_mpe'].max() * 1.1],
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="center",
            x=0.5
        )
    )

    # Display the figure
    fig.show()

# Print statistics of pose evaluation values if available
if pose_col:
    print(f"\nStatistics for {pose_col}:")
    print(f"Minimum: {df[pose_col].min():.2f}")
    print(f"Maximum: {df[pose_col].max():.2f}")
    print(f"Mean: {df[pose_col].mean():.2f}")
    print(f"Median: {df[pose_col].median():.2f}")
    print(f"Count of values < 0.85: {len(df[df[pose_col] < 0.85])}")
    print(f"Count of values ≥ 0.85: {len(df[df[pose_col] >= 0.85])}")
    
    # Breakdown by pond type
    print("\nBreakdown by pond type:")
    for pond_type in df['Pond_Type'].unique():
        pond_df = df[df['Pond_Type'] == pond_type]
        low_count = len(pond_df[pond_df[pose_col] < 0.85])
        good_count = len(pond_df[pond_df[pose_col] >= 0.85])
        total = len(pond_df)
        print(f"{pond_type} pond:")
        print(f"  Values < 0.85: {low_count} ({low_count/total*100:.1f}%)")
        print(f"  Values ≥ 0.85: {good_count} ({good_count/total*100:.1f}%)")

Number of rows before filtering: 117
Number of rows after filtering: 112



Statistics for pose_eval_iou:
Minimum: 0.92
Maximum: 1.00
Mean: 0.99
Median: 1.00
Count of values < 0.85: 0
Count of values ≥ 0.85: 112

Breakdown by pond type:
circle_female pond:
  Values < 0.85: 0 (0.0%)
  Values ≥ 0.85: 89 (100.0%)
circle_male pond:
  Values < 0.85: 0 (0.0%)
  Values ≥ 0.85: 13 (100.0%)
square pond:
  Values < 0.85: 0 (0.0%)
  Values ≥ 0.85: 10 (100.0%)


In [42]:

#show median mpe now
print('median mpe:')
print(df.groupby('Pond_Type')['min_mpe'].median())

#show median mpe now
print('median mpe:')
print(df.groupby('Pond_Type')['min_mpe'].median())

median mpe:
Pond_Type
circle_female     4.767052
circle_male      12.110762
square            4.604901
Name: min_mpe, dtype: float64
median mpe:
Pond_Type
circle_female     4.767052
circle_male      12.110762
square            4.604901
Name: min_mpe, dtype: float64


In [44]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go



# Calculate minimum percentage error (MPE) if not already in dataframe
if 'min_mpe' not in df.columns:
    df['min_mpe'] = df[['MPError_fov_min', 'MPError_fov_max', 'MPError_fov_median']].min(axis=1)

# Calculate pixel differences
df['pred_pixels_diff_1'] = abs(df['pred_Distance_pixels'] - df['Length_1_pixels'])
df['pred_pixels_diff_2'] = abs(df['pred_Distance_pixels'] - df['Length_2_pixels'])
df['pred_pixels_diff_3'] = abs(df['pred_Distance_pixels'] - df['Length_3_pixels'])


# Define image dimensions
IMAGE_WIDTH = 5312
IMAGE_HEIGHT = 2988

# Calculate diagonal length of image in pixels
IMAGE_DIAGONAL = np.sqrt(IMAGE_WIDTH**2 + IMAGE_HEIGHT**2)

# Calculate pixel differences as percentage of image diagonal
df['pred_pixels_diff_1_pct'] = df['pred_pixels_diff_1'] / IMAGE_DIAGONAL * 100
df['pred_pixels_diff_2_pct'] = df['pred_pixels_diff_2'] / IMAGE_DIAGONAL * 100 
df['pred_pixels_diff_3_pct'] = df['pred_pixels_diff_3'] / IMAGE_DIAGONAL * 100

# Calculate minimum percentage pixel difference
df['min_pixel_diff_pct'] = df[['pred_pixels_diff_1_pct', 'pred_pixels_diff_2_pct', 'pred_pixels_diff_3_pct']].min(axis=1)

# Print statistics for percentage pixel differences
print("\nPixel differences as percentage of image diagonal:")
print(f"Minimum: {df['min_pixel_diff_pct'].min():.2f}%")
print(f"Maximum: {df['min_pixel_diff_pct'].max():.2f}%") 
print(f"Mean: {df['min_pixel_diff_pct'].mean():.2f}%")
print(f"Median: {df['min_pixel_diff_pct'].median():.2f}%")




# Calculate minimum pixel difference
df['min_pixel_diff'] = df[['pred_pixels_diff_1', 'pred_pixels_diff_2', 'pred_pixels_diff_3']].min(axis=1)

# Print basic statistics
print(f"Minimum pixel difference: {df['min_pixel_diff'].min():.2f}")
print(f"Maximum pixel difference: {df['min_pixel_diff'].max():.2f}")
print(f"Mean pixel difference: {df['min_pixel_diff'].mean():.2f}")
print(f"Median pixel difference: {df['min_pixel_diff'].median():.2f}")

# Define color categories for pixel differences
pixel_diff_bins = [0, 5, 10, 20, float('inf')]
pixel_diff_labels = ['<5px', '5-10px', '10-20px', '>20px']
pixel_diff_colors = {
    '<5px': '#2ecc71',    # Green
    '5-10px': '#f1c40f',  # Yellow
    '10-20px': '#e74c3c', # Red
    '>20px': '#9b59b6'    # Purple
}

# Create separate figure for each pond type
for pond_type in df['Pond_Type'].unique():
    # Filter data for this pond type
    pond_df = df[df['Pond_Type'] == pond_type].copy()
    
    # Categorize pixel differences
    pond_df['pixel_diff_category'] = pd.cut(
        pond_df['min_pixel_diff'],
        bins=pixel_diff_bins,
        labels=pixel_diff_labels
    )
    
    # Sort by min MPE for consistent visualization
    pond_df = pond_df.sort_values(by='min_mpe')
    
    # Create figure
    fig = go.Figure()
    
    # Create y-values to spread points vertically
    y_values = np.linspace(0.1, 0.9, len(pond_df))
    
    # Create scatter plot for each pixel difference category
    for category in pixel_diff_labels:
        cat_df = pond_df[pond_df['pixel_diff_category'] == category]
        if len(cat_df) > 0:
            # Get indices for y values
            indices = [list(pond_df.index).index(idx) for idx in cat_df.index]
            cat_y_values = [y_values[i] for i in indices]
            
            # Add scatter trace
            fig.add_trace(
                go.Scatter(
                    x=cat_df['min_mpe'],
                    y=cat_y_values,
                    mode='markers',
                    marker=dict(
                        color=pixel_diff_colors[category],
                        size=10,
                        line=dict(width=1, color='black')
                    ),
                    name=category,
                    hovertemplate=
                    "ID: %{customdata[0]}<br>" + 
                    "Image: %{customdata[1]}<br>" +
                    "Error (%): %{x:.1f}%<br>" + 
                    "Pixel Diff: %{customdata[2]:.1f}px<br>" +
                    "<extra></extra>",
                    customdata=cat_df[['PrawnID', 'Label', 'min_pixel_diff']].values
                )
            )
    
    # Add vertical lines at 5% and 10% error boundaries
    fig.add_shape(
        type="line", x0=5, x1=5, y0=0, y1=1,
        line=dict(color="gray", width=1, dash="dash")
    )
    fig.add_shape(
        type="line", x0=10, x1=10, y0=0, y1=1,
        line=dict(color="gray", width=1, dash="dash")
    )
    
    # Add annotations
    fig.add_annotation(x=5, y=0.95, text="5%", showarrow=False)
    fig.add_annotation(x=10, y=0.95, text="10%", showarrow=False)

    # Update layout
    fig.update_layout(
        height=500,
        width=800,
        title_text=f"Error Distribution by Pixel Difference - {pond_type} Pond",
        template="plotly_white",
        xaxis_title="Min MPE (%)",
        yaxis_title="",
        yaxis_showticklabels=False,
        xaxis_range=[0, df['min_mpe'].max() * 1.1],
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="center",
            x=0.5
        )
    )

    # Display the figure
    fig.show()

# Print statistics by pond type
print("\nPixel difference statistics by pond type:")
for pond_type in df['Pond_Type'].unique():
    pond_df = df[df['Pond_Type'] == pond_type]
    print(f"\n{pond_type} pond:")
    print(f"  Min: {pond_df['min_pixel_diff'].min():.2f}px")
    print(f"  Max: {pond_df['min_pixel_diff'].max():.2f}px")
    print(f"  Mean: {pond_df['min_pixel_diff'].mean():.2f}px")
    print(f"  Median: {pond_df['min_pixel_diff'].median():.2f}px")
    
    # Distribution of points across pixel difference categories
    pixel_categories = pd.cut(
        pond_df['min_pixel_diff'],
        bins=pixel_diff_bins,
        labels=pixel_diff_labels
    ).value_counts()
    
    print("  Distribution:")
    for category, count in pixel_categories.items():
        percentage = count / len(pond_df) * 100
        print(f"    {category}: {count} points ({percentage:.1f}%)")


Pixel differences as percentage of image diagonal:
Minimum: 0.00%
Maximum: 1.75%
Mean: 0.34%
Median: 0.22%
Minimum pixel difference: 0.16
Maximum pixel difference: 106.95
Mean pixel difference: 20.94
Median pixel difference: 13.38



Pixel difference statistics by pond type:

circle_female pond:
  Min: 0.16px
  Max: 85.70px
  Mean: 21.08px
  Median: 13.88px
  Distribution:
    >20px: 33 points (37.1%)
    10-20px: 20 points (22.5%)
    <5px: 18 points (20.2%)
    5-10px: 18 points (20.2%)

circle_male pond:
  Min: 1.99px
  Max: 43.25px
  Mean: 17.22px
  Median: 15.79px
  Distribution:
    10-20px: 4 points (30.8%)
    >20px: 4 points (30.8%)
    5-10px: 3 points (23.1%)
    <5px: 2 points (15.4%)

square pond:
  Min: 0.83px
  Max: 106.95px
  Mean: 24.49px
  Median: 10.63px
  Distribution:
    5-10px: 3 points (30.0%)
    >20px: 3 points (30.0%)
    <5px: 2 points (20.0%)
    10-20px: 2 points (20.0%)


In [46]:
df.columns


Index(['Label', 'PrawnID', 'Avg_Length', 'Std_Length', 'Uncertainty',
       'Length_1', 'Length_2', 'Length_3', 'BoundingBox_1', 'BoundingBox_2',
       'BoundingBox_3', 'Angle_1', 'Angle_2', 'Angle_3', 'Scale_1', 'Scale_2',
       'Scale_3', 'Global_Uncertainty', 'Length_ground_truth_annotation(mm)',
       'Length_ground_truth_annotation_pixels', 'pred_Distance_pixels', 'id',
       'Length_fov(mm)', 'Height(mm)', 'focal_RealLength(cm)',
       'Length_1_pixels', 'Length_2_pixels', 'Length_3_pixels', 'Pond_Type',
       'Euclidean_Distance', 'MPError_fov_min', 'MPError_fov_max',
       'MPError_fov_median', 'AbsError_fov_min', 'AbsError_fov_max',
       'AbsError_fov_median', 'AbsError_focal_min', 'AbsError_focal_max',
       'AbsError_focal_median', 'MPError_focal_min', 'MPError_focal_max',
       'MPError_focal_median', 'Error_distance_mm_ground_min',
       'Error_distance_mm_ground_max', 'Error_distance_mm_ground_median',
       'Error_percentage_distance_mm_ground_min',
       

In [54]:
import pandas as pd
import numpy as np
import plotly.graph_objects as go



# Calculate minimum percentage error (MPE) if not already in dataframe
if 'min_mpe' not in df.columns:
    df['min_mpe'] = df[['MPError_fov_min', 'MPError_fov_max', 'MPError_fov_median']].min(axis=1)

# Calculate differences between ground truth annotation and expert measurements in pixels
if 'Length_ground_truth_annotation_pixels' in df.columns:
    # Calculate minimum expert pixels
    df['min_expert_pixels'] = df[['Length_1_pixels', 'Length_2_pixels', 'Length_3_pixels']].min(axis=1)
    
    # Get the expert measurement that gave the min MPE
    # Get the expert measurement that gave the min MPE
    min_mpe_mask = df[['MPError_fov_min', 'MPError_fov_max', 'MPError_fov_median']].eq(df['min_mpe'], axis=0)

    # Extract the index numbers and handle potential NaN values
    extracted_numbers = min_mpe_mask.idxmax(axis=1).str.extract('(\d+)')
    # Fill NaN with a default value (e.g., '1') and then convert to int
    expert_idx = extracted_numbers[0].fillna('1').astype(int)

    # Calculate ground truth difference using that expert measurement
    df['gt_diff'] = abs(df['Length_ground_truth_annotation_pixels'] - 
                    df.apply(lambda x: x[f'Length_{expert_idx[x.name]}_pixels'], axis=1))
else:
    print("Warning: 'Length_ground_truth_annotation_pixels' column not found in the dataset.")
    print(f"Available columns: {df.columns.tolist()}")
    exit(1)

# Print basic statistics
print(f"Minimum ground truth difference: {df['gt_diff'].min():.2f} pixels")
print(f"Maximum ground truth difference: {df['gt_diff'].max():.2f} pixels")
print(f"Mean ground truth difference: {df['gt_diff'].mean():.2f} pixels")
print(f"Median ground truth difference: {df['gt_diff'].median():.2f} pixels")

# Define color categories for pixel differences
# Adjust the bins based on the distribution of your data
pixel_diff_bins = [0, 5, 10, 20, float('inf')]
pixel_diff_labels = ['<5px', '5-10px', '10-20px', '>20px']
pixel_diff_colors = {
    '<5px': '#2ecc71',    # Green
    '5-10px': '#f1c40f',  # Yellow
    '10-20px': '#e74c3c', # Red
    '>20px': '#9b59b6'    # Purple
}

# Create separate figure for each pond type
for pond_type in df['Pond_Type'].unique():
    # Filter data for this pond type
    pond_df = df[df['Pond_Type'] == pond_type].copy()
    
    # Categorize pixel differences
    pond_df['gt_diff_category'] = pd.cut(
        pond_df['gt_diff'],
        bins=pixel_diff_bins,
        labels=pixel_diff_labels
    )
    
    # Sort by min MPE for consistent visualization
    pond_df = pond_df.sort_values(by='min_mpe')
    
    # Create figure
    fig = go.Figure()
    
    # Create y-values to spread points vertically
    y_values = np.linspace(0.1, 0.9, len(pond_df))
    
    # Create scatter plot for each pixel difference category
    for category in pixel_diff_labels:
        cat_df = pond_df[pond_df['gt_diff_category'] == category]
        if len(cat_df) > 0:
            # Get indices for y values
            indices = [list(pond_df.index).index(idx) for idx in cat_df.index]
            cat_y_values = [y_values[i] for i in indices]
            
            # Add scatter trace
            fig.add_trace(
                go.Scatter(
                    x=cat_df['min_mpe'],
                    y=cat_y_values,
                    mode='markers',
                    marker=dict(
                        color=pixel_diff_colors[category],
                        size=10,
                        line=dict(width=1, color='black')
                    ),
                    name=category,
                    hovertemplate=
                    "ID: %{customdata[0]}<br>" + 
                    "Image: %{customdata[1]}<br>" +
                    "Error (%): %{x:.1f}%<br>" + 
                    "Ground Truth Diff: %{customdata[2]:.1f}px<br>" +
                    "GT Pixel Length: %{customdata[3]:.1f}px<br>" +
                    "Expert Pixels (min): %{customdata[4]:.1f}px<br>" +
                    "<extra></extra>",
                    customdata=cat_df[['PrawnID', 'Label', 'gt_diff', 
                                     'Length_ground_truth_annotation_pixels', 
                                     'min_expert_pixels']].values
                )
            )
    
    # Add vertical lines at 5% and 10% error boundaries
    fig.add_shape(
        type="line", x0=5, x1=5, y0=0, y1=1,
        line=dict(color="gray", width=1, dash="dash")
    )
    fig.add_shape(
        type="line", x0=10, x1=10, y0=0, y1=1,
        line=dict(color="gray", width=1, dash="dash")
    )
    
    # Add annotations
    fig.add_annotation(x=5, y=0.95, text="5%", showarrow=False)
    fig.add_annotation(x=10, y=0.95, text="10%", showarrow=False)

    # Update layout
    fig.update_layout(
        height=500,
        width=800,
        title_text=f"Error Distribution by Ground Truth Annotation Difference - {pond_type} Pond",
        template="plotly_white",
        xaxis_title="Min MPE (%)",
        yaxis_title="",
        yaxis_showticklabels=False,
        xaxis_range=[0, df['min_mpe'].max() * 1.1],
        legend=dict(
            orientation="h",
            yanchor="bottom",
            y=1.02,
            xanchor="center",
            x=0.5
        )
    )

    # Display the figure
    fig.show()

# Additional detailed analysis: box plots of ground truth differences by expert
fig = go.Figure()

# Create box plots for each expert vs ground truth
experts = ['Length_1_pixels', 'Length_2_pixels', 'Length_3_pixels']
expert_names = ['Expert 1', 'Expert 2', 'Expert 3']

for i, expert in enumerate(experts):
    diff_column = f'gt_diff_{i+1}'
    fig.add_trace(
        go.Box(
            y=df[diff_column],
            name=expert_names[i],
            boxpoints='all',
            jitter=0.3,
            pointpos=-1.8,
            marker=dict(
                color='rgba(0,0,0,0.3)',
                size=3
            ),
            hovertemplate=
            "Difference: %{y:.1f}px<br>" +
            "<extra></extra>"
        )
    )

fig.update_layout(
    title_text="Pixel Differences: Ground Truth vs Expert Measurements",
    yaxis_title="Pixel Difference",
    boxmode='group',
    template="plotly_white",
    height=500,
    width=800
)

fig.show()

# Print statistics by pond type
print("\nGround truth difference statistics by pond type:")
for pond_type in df['Pond_Type'].unique():
    pond_df = df[df['Pond_Type'] == pond_type]
    print(f"\n{pond_type} pond:")
    print(f"  Min: {pond_df['gt_diff'].min():.2f}px")
    print(f"  Max: {pond_df['gt_diff'].max():.2f}px")
    print(f"  Mean: {pond_df['gt_diff'].mean():.2f}px")
    print(f"  Median: {pond_df['gt_diff'].median():.2f}px")
    
    # Distribution of points across pixel difference categories
    gt_categories = pd.cut(
        pond_df['gt_diff'],
        bins=pixel_diff_bins,
        labels=pixel_diff_labels
    ).value_counts()
    
    print("  Distribution:")
    for category, count in gt_categories.items():
        percentage = count / len(pond_df) * 100
        print(f"    {category}: {count} points ({percentage:.1f}%)")
        
# Additional analysis: correlation between ground truth difference and min MPE
correlation = df['gt_diff'].corr(df['min_mpe'])
print(f"\nCorrelation between ground truth pixel difference and min MPE: {correlation:.4f}")

# Create a scatter plot of ground truth difference vs min MPE
fig = go.Figure()

for pond_type in df['Pond_Type'].unique():
    pond_df = df[df['Pond_Type'] == pond_type]
    
    fig.add_trace(
        go.Scatter(
            x=pond_df['gt_diff'],
            y=pond_df['min_mpe'],
            mode='markers',
            name=pond_type,
            marker=dict(
                size=8,
                opacity=0.7
            ),
            hovertemplate=
            "GT Diff: %{x:.1f}px<br>" +
            "Min MPE: %{y:.1f}%<br>" +
            "ID: %{customdata[0]}<br>" +
            "Image: %{customdata[1]}<br>" +
            "<extra></extra>",
            customdata=pond_df[['PrawnID', 'Label']].values
        )
    )

fig.update_layout(
    title_text="Ground Truth Pixel Difference vs Min MPE",
    xaxis_title="Ground Truth Pixel Difference",
    yaxis_title="Min MPE (%)",
    template="plotly_white",
    height=500,
    width=800
)

# Add a trend line
x_range = np.linspace(df['gt_diff'].min(), df['gt_diff'].max(), 100)
slope, intercept = np.polyfit(df['gt_diff'], df['min_mpe'], 1)
fig.add_trace(
    go.Scatter(
        x=x_range,
        y=slope * x_range + intercept,
        mode='lines',
        name=f'Trend (r={correlation:.2f})',
        line=dict(color='black', dash='dash')
    )
)

fig.show()

Minimum ground truth difference: 0.07 pixels
Maximum ground truth difference: 474.18 pixels
Mean ground truth difference: 37.03 pixels
Median ground truth difference: 23.91 pixels



Ground truth difference statistics by pond type:

circle_female pond:
  Min: 0.43px
  Max: 474.18px
  Mean: 36.72px
  Median: 22.52px
  Distribution:
    >20px: 48 points (53.9%)
    10-20px: 21 points (23.6%)
    <5px: 11 points (12.4%)
    5-10px: 9 points (10.1%)

circle_male pond:
  Min: 2.22px
  Max: 64.84px
  Mean: 25.75px
  Median: 22.68px
  Distribution:
    >20px: 8 points (61.5%)
    10-20px: 3 points (23.1%)
    <5px: 1 points (7.7%)
    5-10px: 1 points (7.7%)

square pond:
  Min: 0.07px
  Max: 167.64px
  Mean: 54.44px
  Median: 40.73px
  Distribution:
    >20px: 8 points (80.0%)
    <5px: 2 points (20.0%)
    5-10px: 0 points (0.0%)
    10-20px: 0 points (0.0%)

Correlation between ground truth pixel difference and min MPE: -0.0941


In [14]:
#length range between min and max
print('length range between min and max:')
print(df.groupby('Pond_Type').agg({
    'Length_1': ['min', 'max'],
    'Length_2': ['min', 'max'], 
    'Length_3': ['min', 'max']
}).round(2))


length range between min and max:
          Length_1         Length_2         Length_3        
               min     max      min     max      min     max
Pond_Type                                                   
car          89.03  144.13    86.33  137.24    95.56  140.43
left         36.90  135.61    34.40  134.93    39.16  126.32
right        68.78  169.07    63.03  171.18    70.58  166.31


In [18]:
#median and std of the length

import scipy.stats as stats

from scipy.stats import median_abs_deviation

print('median and median absolute deviation of the length:')
print(df.groupby('Pond_Type').agg({
    'Length_1': [lambda x: np.median(x), lambda x: median_abs_deviation(x, scale='normal')],
    'Length_2': [lambda x: np.median(x), lambda x: median_abs_deviation(x, scale='normal')],
    'Length_3': [lambda x: np.median(x), lambda x: median_abs_deviation(x, scale='normal')]
}).round(2))



median and median absolute deviation of the length:
            Length_1              Length_2              Length_3           
          <lambda_0> <lambda_1> <lambda_0> <lambda_1> <lambda_0> <lambda_1>
Pond_Type                                                                  
car           109.16      11.85     111.73      18.69     113.49      20.64
left           64.54      22.74      68.79      19.66      71.72      19.98
right         113.84      18.84     112.24      20.75     120.17      20.82


In [19]:
df['Mean_Length'] = df[['Length_1', 'Length_2', 'Length_3']].mean(axis=1)
df['Std_Dev_Length'] = df[['Length_1', 'Length_2', 'Length_3']].std(axis=1)

# Derive overall uncertainty by pond type
overall_stats = df.groupby('Pond_Type').agg({
    'Mean_Length': 'mean',
    'Std_Dev_Length': 'mean'
}).rename(columns={
    'Mean_Length': 'Overall_Mean_Length',
    'Std_Dev_Length': 'Overall_Uncertainty'
}).reset_index()

# Print per-prawn uncertainty
print("Per-Prawn Uncertainty:")
# print(df[['Pond_Type', 'Mean_Length', 'Std_Dev_Length']].round(2))

# Print overall uncertainty by pond type
print("\nOverall Uncertainty by Pond Type:")
print(overall_stats.round(2))

Per-Prawn Uncertainty:

Overall Uncertainty by Pond Type:
  Pond_Type  Overall_Mean_Length  Overall_Uncertainty
0       car               114.17                 7.11
1      left                73.02                 5.27
2     right               116.34                 7.53


In [48]:
fig = px.scatter(df,color='PondType', x='min_mpe', y='min_mpe_pixels',hover_data=['Label', 'PrawnID'])
#add title
fig.update_layout(title=f'min_mpe vs. min_mpe_pixels')

#save the plot to png
fig.write_image(f'full body_min_mpe vs. min_mpe_pixels.png')
# Add a trendline
fig.show()

In [47]:
#relationship between min_mpe and min_mpe_of_pixels
for pond_type in df['PondType'].unique():

    fig = px.scatter(df[df['PondType'] == pond_type], x='min_mpe', y='min_mpe_pixels',hover_data=['Label', 'PrawnID'])
    #add title
    fig.update_layout(title=f'min_mpe vs. min_mpe_pixels {pond_type}')

    #save the plot to png
    fig.write_image(f'full body_min_mpe vs. min_mpe_pixels {pond_type}.png')
    # Add a trendline
    fig.show()


In [11]:
fig = px.scatter(df, x='Height_mm', y='min_mae', hover_data=['Label', 'PrawnID'])
#add title
fig.update_layout(title='min_mae vs. Height')

#save the plot to png
fig.write_image('full body_ min_mae vs. Height.png')
# Add a trendline
fig.show()

In [15]:
# a graph showing overestimation and underestimation of the length of the prawn
df['overestimation'] = df['Length_FOV_box'] - df['expert_max']
#show the overestimation of the length of the prawn
fig = px.scatter(df, x='Height_mm', y='overestimation', hover_data=['Label', 'PrawnID'])
#add title
fig.update_layout(title='overestimation vs. Height')

#save the plot to png
fig.write_image('full body_ overestimation vs. Height.png')
# Add a trendline
fig.show()



In [23]:
#groupby pond type unique label
df.groupby('PondType')['Label'].nunique()

PondType
circular-big      33
circular-small     5
square-female     13
Name: Label, dtype: int64

In [11]:
import plotly.express as px





df['min_mpe_length-'] = df.apply(lambda row: row['Length_FOV_box'] - row['Length_FOV_box'] * row['min_mpe']/100, axis=1)
                                
df['min_mpe_length+']= df.apply(lambda row: row['Length_FOV_box'] + row['Length_FOV_box'] * row['min_mpe']/100, axis=1)

df['min_mae_length-'] = df.apply(lambda row: row['Length_FOV_box'] -  row['min_mae'], axis=1)
df['min_mae_length+'] = df.apply(lambda row: row['Length_FOV_box'] +  row['min_mae'], axis=1)




def get_closest_length_and_distance(row, target_length1,true_length2):
    lengths = [row['Length_1'], row['Length_2'], row['Length_3']]
    closest_length_1 = min(lengths, key=lambda x: abs(x - target_length1))
    distance_1 = abs(closest_length_1 - target_length1)
    closest_length_2 = min(lengths, key=lambda x: abs(x - true_length2))
    distance_2 = abs(closest_length_2 - true_length2)

    if distance_1 < distance_2:
        closest_length = closest_length_1
        distance = distance_1
    else:
        closest_length = closest_length_2
        distance = distance_2


    return closest_length, distance

# Apply the function to find the closest length and distance to min_mpe_length-
df['closest_length_min_mpe'] = df.apply(
    lambda row: (get_closest_length_and_distance(row, row['min_mpe_length-'],row['min_mpe_length+'])), axis=1
)

df['closest_length_min_mae'] = df.apply( lambda row: (get_closest_length_and_distance(row, row['min_mae_length-'],row['min_mae_length+'])), axis=1)

# Apply the function to find the closest length and distance to min_mpe_length+
df['closest_length_min_mae_value']=df['closest_length_min_mae'].apply(lambda x: x[0] if isinstance(x, tuple) else x)
# Assuming df contains the columns 'predicted_length' and 'true_length'
#take the overestimation from the min_percentage_error length
# find the true length when min_mpe is used

#min_mpe_length will be decided by the min_mpe percentage error 


df['closest_length_min_mpe_value'] = df['closest_length_min_mpe'].apply(lambda x: x[0] if isinstance(x, tuple) else x)

# Calculate overestimation
df['overestimation'] = df['Length_FOV_box'] - df['closest_length_min_mae_value']














                                 #approximatly equal to length_1 

# df['overestimation'] = df['Length_FOV_box'] - df['closest_length_min_mpe']






for pond_type in df['PondType'].unique():
    fig = px.histogram(df[df['PondType'] == pond_type], x='overestimation', nbins=30, marginal="box", title=f"Histogram of Residuals {pond_type} Overestimation ",hover_data=['Label', 'PrawnID'])
    fig.update_layout(
    xaxis_title=f"Residuals {pond_type} (Overestimation vs Underestimation)",
    yaxis_title="Frequency",
    )
    # Add a vertical line at 0 to show the separation between overestimation and underestimation
    fig.add_vline(x=0, line_dash="dash", line_color="red")

    # Save the plot to a file
    fig.write_image(f'histogram_residuals {pond_type}.png')

    fig.show()

# df['underestimation'] = df['Length_FOV_box'] - df['expert_min']

# for pond_type in df['PondType'].unique():
#     fig = px.histogram(df[df['PondType'] == pond_type], x='underestimation', nbins=30, marginal="box", title=f"Histogram of Residuals {pond_type}  Underestimation)",hover_data=['Label', 'PrawnID'])
#     fig.update_layout(
#     xaxis_title=f"Residuals {pond_type} (Overestimation vs Underestimation)",
#     yaxis_title="Frequency",
#     )
#     # Add a vertical line at 0 to show the separation between overestimation and underestimation
#     fig.add_vline(x=0, line_dash="dash", line_color="red")

#     # Save the plot to a file
#     fig.write_image(f'histogram_residuals {pond_type}.png')

#     fig.show()

# Calculate overestimation and underestimation counts




In [12]:
import plotly.graph_objects as go


#scatter plot between length_fov and closest_length_min_mae_value
fig = px.scatter(df,color='PondType', x='Length_box_focal', y='closest_length_min_mae_value', hover_data=['Label', 'PrawnID'])
#add title
fig.update_layout(title=f'pinhole vs. closest_length_min_mae_value')
fig.add_trace(go.Scatter(x=[0, 200], y=[0, 200], mode='lines', name='line'))

#save the plot to png
fig.write_image(f'full focal vs. closest_length_min_mae_value.png')
#add a trendline

#add line


fig.show()
#

In [18]:
overestimations = len(df[df['overestimation'] > 0])
underestimations = len(df[df['overestimation'] < 0])

# Create a bar plot
fig = px.bar(
    x=['Overestimations', 'Underestimations'], 
    y=[overestimations, underestimations],
    title="Overestimations vs Underestimations"
)
fig.update_layout(
    yaxis_title="Count"
)
# Save the plot to a file
fig.write_image('overestimations_vs_underestimations.png')

fig.show()

In [40]:
    
    
    
for pond_type in df['PondType'].unique():    


    fig = px.ecdf(df[df['PondType'] == pond_type], x="overestimation", title=f"Cumulative Distribution of Residuals (CDF) {pond_type}",hover_data=['Label', 'PrawnID'])
    fig.update_layout(
        xaxis_title="Residuals (Predicted - True)"
    )
    # Add a vertical line at 0 to show the separation between overestimation and underestimation
    fig.add_vline(x=0, line_dash="dash", line_color="red")

    fig.show()


In [49]:
#min_mpe_focal_length from MPError_focal_min, MPError_focal_max, MPError_focal_median
# min_mae_focal_length from AbsError_focal_min, AbsError_focal_max, AbsError_focal_median

df['min_mpe_focal_length']=df[['Error_percentage_max_box_focal','Error_percentage_min_box_focal','Error_percentage_median_box_focal']].min(axis=1)

df['min_mae_focal_length']=df[['Abs_error_max_box_focal','Abs_error_min_box_focal','Abs_error_median_box_focal']].min(axis=1)

df['max_mpe_focal_length']=df[['Error_percentage_max_box_focal','Error_percentage_min_box_focal','Error_percentage_median_box_focal']].max(axis=1)

df['max_mae_focal_length']=df[['Abs_error_max_box_focal','Abs_error_min_box_focal','Abs_error_median_box_focal']].max(axis=1)



#print title of the prints
print('pond type count:')
print(df['PondType'].value_counts())

print('pond type mean:')
print(df.groupby('PondType')['Avg_Length'].mean())

print('pond type std:')
print(df.groupby('PondType')['Avg_Length'].std())


print ('min_mpe fov median:')
print(df.groupby('PondType')['min_mpe'].median())
#mean and std for each pond type of the lengt_1, length_2, length_

#print min_mae median and std for each pond type

print('min_mae fov median:')
print(df.groupby('PondType')['min_mae'].median())



print('max_mpe fov median:')
print(df.groupby('PondType')['max_mpe'].median())

print('max_mae fov median:')
print(df.groupby('PondType')['max_mae'].median())


print('min_mpe focal length median:')
print(df.groupby('PondType')['min_mpe_focal_length'].median())

print('max_mpe focal length median:')
print(df.groupby('PondType')['max_mpe_focal_length'].median()) 

print('min_mae focal length median:')
print(df.groupby('PondType')['min_mae_focal_length'].median())

print('max_mae focal length median:')   
print(df.groupby('PondType')['max_mae_focal_length'].median())








pond type count:
PondType
circular-big      93
circular-small    15
square-female     14
Name: count, dtype: int64
pond type mean:
PondType
circular-big      116.341756
circular-small     73.021133
square-female     119.587619
Name: Avg_Length, dtype: float64
pond type std:
PondType
circular-big      18.716363
circular-small    23.583171
square-female     17.805035
Name: Avg_Length, dtype: float64
min_mpe fov median:
PondType
circular-big       8.873005
circular-small    24.395622
square-female      1.922570
Name: min_mpe, dtype: float64
min_mae fov median:
PondType
circular-big      11.390734
circular-small    20.878955
square-female      2.386082
Name: min_mae, dtype: float64
max_mpe fov median:
PondType
circular-big      23.144713
circular-small    47.090420
square-female     12.111046
Name: max_mpe, dtype: float64
max_mae fov median:
PondType
circular-big      24.212128
circular-small    30.365965
square-female     15.193373
Name: max_mae, dtype: float64
min_mpe focal length median

In [14]:
#mean difference between min max of Length_1, Length_2, Length_3 for each pond type
print('mean difference between min max for each pond type:')
print(df.groupby('PondType')['diff_max_min'].mean())

mean difference between min max for each pond type:
PondType
circular-big      14.402247
circular-small     9.966533
square-female     14.834071
Name: diff_max_min, dtype: float64


In [16]:
#print uncertainty of the mean 
print('uncertainty of the mean:')
print(df.groupby('Pond_Type')['Std_Length'].mean()/np.sqrt(df.groupby('Pond_Type')['Avg_Length'].count()))

uncertainty of the mean:
Pond_Type
circular big male      0.261743
circular small male    0.649817
square-femal           0.428996
dtype: float64


In [ ]:
X = df[['camera_distance', 'pose_metric_1', 'pose_metric_2']]  # Add other pose metrics if available
X = sm.add_constant(X)
y = df['diff']
model = sm.OLS(y, X).fit()
print(model.summary())




melted_df = df.melt(id_vars=['Label','PrawnID'], value_vars=['Length_1', 'Length_2', 'Length_3'],
                   var_name='rater', value_name='measurement')
icc = pg.intraclass_corr(data=melted_df, targets=['Label','PrawnID'], raters='rater', ratings='measurement')
print(icc)


# Correlation Analysis
pearson_corr, pearson_p = pearsonr(df['keypoint_mm'], df['expert_mean'])
spearman_corr, spearman_p = spearmanr(df['keypoint_mm'], df['expert_mean'])
print(f"Pearson Correlation: {pearson_corr:.3f}, p-value: {pearson_p:.3e}")
print(f"Spearman Correlation: {spearman_corr:.3f}, p-value: {spearman_p:.3e}")

mean_measurements = (df['keypoint_mm'] + df['expert_mean']) / 2
diff_measurements = df['diff']
md = np.mean(diff_measurements)
sd = np.std(diff_measurements)

plt.figure(figsize=(10, 6))
plt.scatter(mean_measurements, diff_measurements, alpha=0.5)
plt.axhline(md, color='gray', linestyle='--', label=f'Mean Diff = {md:.2f} mm')
plt.axhline(md + 1.96*sd, color='red', linestyle='--', label=f'+1.96 SD = {md + 1.96*sd:.2f} mm')
plt.axhline(md - 1.96*sd, color='red', linestyle='--', label=f'-1.96 SD = {md - 1.96*sd:.2f} mm')
plt.xlabel('Mean of Measurements (mm)')
plt.ylabel('Difference (Keypoint - Expert) (mm)')
plt.title('Bland-Altman Plot')
plt.legend()
plt.show()



stat, p = shapiro(df['diff'])
print(f'Shapiro-Wilk Test: stat={stat:.3f}, p-value={p:.3e}')

# Homogeneity of Variance
# Assuming 'camera_distance_category' is already defined as in the visualization section
groups = df.groupby('camera_distance_category')['diff'].apply(list)
stat, p = levene(*groups)
print(f"Levene’s Test: stat={stat:.3f}, p-value={p:.3e}")
